# Missing Category imputation with Scikit-learn -- SimpleImputer

### Datasets:
- House Prices dataset


### Content:

1. SimpleImputer on the entire dataset.
2. SimpleImputer: different procedures on different features.

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split

### 1. SimpleImputer on the entire dataset

In [2]:
# load the dataset with a few columns for the demonstration

cols_to_use = ['BsmtQual', 'FireplaceQu', 'SalePrice']

data = pd.read_csv('../houseprice.csv', usecols=cols_to_use)
data.head()

,BsmtQual,FireplaceQu,SalePrice
0,Gd,NaN,208500
1,Gd,TA,181500
2,Gd,TA,223500
3,TA,Gd,140000
4,Gd,TA,250000


In [3]:
# inspect missing values

data.isnull().mean()

BsmtQual       0.025342
FireplaceQu    0.472603
SalePrice      0.000000
dtype: float64

The variables LotFrontage, MasVnrArea and GarageYrBlt contain missing data.

In [4]:
# first remove the target from the features
cols_to_use.remove('SalePrice')

X_train, X_test, y_train, y_test = train_test_split(
    data[cols_to_use], 
    data['SalePrice'], 
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((1022, 2), (438, 2))

In [5]:
# the misssing data again
X_train.isnull().mean()

BsmtQual       0.023483
FireplaceQu    0.467710
dtype: float64

In [6]:
# inspect the values of the categorical variable:

# for BsmtQual

X_train['BsmtQual'].unique()

array(['Gd', 'TA', 'Fa', nan, 'Ex'], dtype=object)

In [7]:
# for BsmtQual

X_train['FireplaceQu'].unique()

array([nan, 'Gd', 'TA', 'Fa', 'Po', 'Ex'], dtype=object)

In [8]:
# create imputer
imputer = SimpleImputer(strategy='constant', 
                       fill_value = 'Missing')

# fit the imputer to the train sets
imputer.fit(X_train)

SimpleImputer(fill_value='Missing', strategy='constant')

In [9]:
# demo: the learnt modes like this:
imputer.statistics_

array(['Missing', 'Missing'], dtype=object)

In [10]:
# impute the train and test set
# (returned as a numpy array)

X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

X_train

array([['Gd', 'Missing'],
       ['Gd', 'Gd'],
       ['TA', 'Missing'],
       ...,
       ['Missing', 'Missing'],
       ['Gd', 'TA'],
       ['Gd', 'Missing']], dtype=object)

In [11]:
# encode the train set back to a dataframe

X_train = pd.DataFrame(X_train, columns=cols_to_use)
X_train.head()

,BsmtQual,FireplaceQu
0,Gd,Missing
1,Gd,Gd
2,TA,Missing
3,TA,Missing
4,TA,Missing


In [12]:
X_train['BsmtQual'].unique()

array(['Gd', 'TA', 'Fa', 'Missing', 'Ex'], dtype=object)

In [13]:
X_train.isnull().mean()

BsmtQual       0.0
FireplaceQu    0.0
dtype: float64

### 2. SimpleImputer: different procedures on different features

In [14]:
# load the dataset with both numerical and categorical variables

cols_to_use = [
    'BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt',
    'SalePrice'
]

data = pd.read_csv('../houseprice.csv', usecols=cols_to_use)
data.head()

,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,GarageYrBlt,SalePrice
0,65.0,196.0,Gd,NaN,2003.0,208500
1,80.0,0.0,Gd,TA,1976.0,181500
2,68.0,162.0,Gd,TA,2001.0,223500
3,60.0,0.0,TA,Gd,1998.0,140000
4,84.0,350.0,Gd,TA,2000.0,250000


In [15]:
# first drop the target from the feature list
cols_to_use.remove('SalePrice')

X_train, X_test, y_train, y_test = train_test_split(data[cols_to_use],
                                                    data['SalePrice'],
                                                    test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((1022, 5), (438, 5))

In [16]:
X_train.isnull().mean()

BsmtQual       0.023483
FireplaceQu    0.467710
LotFrontage    0.184932
MasVnrArea     0.004892
GarageYrBlt    0.052838
dtype: float64

impute:
- the numerical variables by the mean
- the categorical variables by the most frequent category.

In [17]:
# lists, indicating which features will be imputed with each method

features_numeric = ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']
features_categoric = ['BsmtQual', 'FireplaceQu']

# put the features list and the transformers together
# using the column transformer

preprocessor = ColumnTransformer(transformers=[
    ('imputer_numeric', SimpleImputer(strategy='mean'), features_numeric),
    ('imputer_categoric', SimpleImputer(strategy='constant', fill_value='Missing'), features_categoric)])

In [18]:
# fit the preprocessor
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('imputer_numeric', SimpleImputer(),
                                 ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']),
                                ('imputer_categoric',
                                 SimpleImputer(fill_value='Missing',
                                               strategy='constant'),
                                 ['BsmtQual', 'FireplaceQu'])])

In [19]:
# demo: statistics for the mean imputer
preprocessor.named_transformers_['imputer_numeric'].statistics_

array([  69.66866747,  103.55358899, 1978.01239669])

In [20]:
# demo: statistics for the constant value imputer
preprocessor.named_transformers_['imputer_categoric'].statistics_

array(['Missing', 'Missing'], dtype=object)

In [21]:
# impute data

X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)

In [22]:
# convert the data back in a dataframe

X_train = pd.DataFrame(X_train,
             columns=features_numeric+features_categoric)

X_train.isnull().mean()

LotFrontage    0.0
MasVnrArea     0.0
GarageYrBlt    0.0
BsmtQual       0.0
FireplaceQu    0.0
dtype: float64